In [ ]:
!pip install scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve, validation_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install imbalanced-learn
#!pip install gplearn
#!pip install dice-ml
!pip install lime
!pip install shap
import lime
import lime.lime_tabular
import shap
import os

In [ ]:
pip install -r requirements.txt

DATA Preparation

In [ ]:
#Extracting sheets,removing unneccessary cells,set dates format

# Load sheets
data_training = pd.read_excel("Training dataset.20150707.xlsx", sheet_name='Training dataset')
data_inspection = pd.read_excel("Training dataset.20150707.xlsx", sheet_name='INSPECTION_RUN')
data_tonnage = pd.read_excel("Training dataset.20150707.xlsx", sheet_name='TONNAGE_SAMPLE_DATA')

# Convert date columns
data_training['TEST_DT'] = pd.to_datetime(data_training['TEST_DT'], format='%Y-%m-%d', errors='coerce')
data_inspection['TEST_DT'] = pd.to_datetime(data_inspection['TEST_DT'], format='%Y-%m-%d', errors='coerce')

# Remove header rows
data_training = data_training.iloc[26:, :]
data_inspection = data_inspection.iloc[3:, :]
data_tonnage = data_tonnage.iloc[4:, :]

# Remove duplicates from training data
data_training = data_training.sort_values(by='DEF_AMPLTD', ascending=True)
data_training = data_training.drop_duplicates(
    subset=['MILEPOST', 'LINE_SEG_NBR', 'TRACK_SDTK_NBR', 'TEST_DT', 'GEO_CAR_NME', 'TSC_CD'],
    keep='last'
).reset_index(drop=True)

# Remove duplicates from tonnage data
data_tonnage = data_tonnage.sort_values(by='TOT_DFLT_MGT', ascending=True)
data_tonnage = data_tonnage.drop_duplicates(
    subset=['LINE_SEG_NBR', 'TRACK_SDTK_NBR', 'YEAR', 'MONTH', 'MILEPOST_START', 'MILEPOST_END'],
    keep='last'
).reset_index(drop=True)


# data training
##display('data_training:')
##display(data_training)
print()
# data inspection
##display('data_inspection:')
##display(data_inspection)
print()
# data tonnage
##display('data_tonnage:')
##display(data_tonnage)
print()

################################################################################################################################################################################################################################################################################################
# variables defenition
##display(" variables Definitions:" )
##display("""***LINE_SEG_NBR: Integer :Every track on the railroad has a unique identifying line segment number. Could be single or double tracks. Using line segment (LINE_SEG_NBR) and mile post (MILPOST_START and MILEPOST_END) you can identify any location on the system.""")
##print()
##display("""***TRACK_SDTK_NBR: CHARACTER : Distingish indivuidaul track segments. Mainline & branch numbers: 0=SINGLE TRACK, 1-9=MULTIPLE MAIN LINES (For example, 1=NORTH MAIN, 2=SOUTH MAIN). Tracks outside of main/branch are referred to as side tracks. 5=SIDING TRACK
##""")
##print()
##display("***TEST_DT: DATE : The date on which testing was performed.")
##print()
##display( "***DEF_NBR:INTEGER:Defect number. Every defect detetected by a Gemoetry car gets a unique id.")
##print()
##display( "***GEO_CAR_NME:CHARACTER:Geometry cars names. Examples of names include: GEO105, GEO505  ETC.")
##print()
##display( "***DEF_PRTY:CHARACTER:Yellow or red.")
##print()
##display( "***DEF_LGTH:INTEGER:Length of defect in feet, as reported by the measurement car.")
##print()
##display( "***DEF_AMPLTD:DECIMAL:Defect amplitude -- maximum size of defect in inches or degrees within defect length.")
##print()
##display( "***TSC_CD:CHARACTER:Track codes including tangent, spiral and curve.")
##print()
##display( "***CLASS:CHARACTER:Class of tracks. All tracks get a number between one and five. Each class represents operating speed limits for passenger and freight traffic. Class one has the lowest speed limit and class five has the highest speed limit.")
##print()
##display( "***TEST_FSPD:CHARACTER:Operating speed for freight trains.")
##print()
##display( "***TEST_PSPD:CHARACTER:Operating speed for passenger trains. If the value = 0, then it means that it does not have passenger traffic.")
##print()
##display( "***DFCT_TYPE:Defect type--the geormetric defect type such as XLEVEL, SURFACE, DIP....")
##print()
##display("***YEAR:SMALLINT:Year for which tonnage is accumulated for.")
##print()
##display("***MONTH:CHARACTER:The month the tonnage was accumulated for this train. Sum of all the the daily mileage for this month.")
##print()
##display("***TOT_CAR_EAST/WEST:INTEGER:Total number of cars traveling east/West.")
##print()
##display("***TOT_TRN_EAST/WEST:INTEGER:Total number of trains traveling east/West.")
##print()
##display("***TOT_DFLT_MGT:DECIMAL:Sum of total gross tons traveling across the section.")

In [ ]:
#Preparing the datasets for matching

# Extract year from test date
data_training['YEAR'] = data_training['TEST_DT'].dt.year.astype('int64')
data_tonnage['YEAR'] = data_tonnage['YEAR'].astype('int64')

# Convert milepost columns to numeric and calculate midpoint
data_tonnage['MILEPOST_START'] = pd.to_numeric(data_tonnage['MILEPOST_START'], errors='coerce')
data_tonnage['MILEPOST_END'] = pd.to_numeric(data_tonnage['MILEPOST_END'], errors='coerce')
data_tonnage['MILEPOST'] = (data_tonnage['MILEPOST_START'] + data_tonnage['MILEPOST_END']) / 2

# Display cleaned data
#display(data_tonnage)
#display(data_training)

In [ ]:
# Merging tonnage and training

# Prepare milepost columns for merging
data_tonnage['MILEPOST'] = pd.to_numeric(data_tonnage['MILEPOST'], errors='coerce').astype('int64')
data_training['MILEPOST'] = pd.to_numeric(data_training['MILEPOST'], errors='coerce').astype('int64')

# Sort by milepost
data_training = data_training.sort_values('MILEPOST')
data_tonnage = data_tonnage.sort_values('MILEPOST')

# Rename to avoid conflicts during merge
data_training.rename(columns={'MILEPOST': 'MILEPOST_training'}, inplace=True)
data_tonnage.rename(columns={'MILEPOST': 'MILEPOST_tonnage'}, inplace=True)

# Merge using merge_asof
merged_data = pd.merge_asof(
    data_training,
    data_tonnage,
    left_on='MILEPOST_training',
    right_on='MILEPOST_tonnage',
    by=['LINE_SEG_NBR', 'TRACK_SDTK_NBR', 'YEAR'],
    direction='nearest',
    suffixes=('_training', '_tonnage')
)

# Clean up columns
merged_data.drop(columns=['Variable Name_tonnage'], inplace=True)
merged_data.rename(columns={
    'Variable Name_training': 'Variable Name',
    'MILEPOST_training': 'MILEPOST'
}, inplace=True)
merged_data.drop(columns=['MILEPOST_tonnage'], inplace=True)

# Define segment ID
merged_data['seg_id'] = (
    merged_data['LINE_SEG_NBR'].astype(str) + '-' +
    merged_data['TRACK_SDTK_NBR'].astype(str) + '-' +
    merged_data['MILEPOST_START'].astype(str) + '-' +
    merged_data['MILEPOST_END'].astype(str)
)
merged_data.insert(0, 'seg_id', merged_data.pop('seg_id'))

# Filter by milepost range with tolerance
tolerance = 0.1
filtered_data_XAI = merged_data[
    (merged_data['MILEPOST'] >= merged_data['MILEPOST_START'] * (1 - tolerance)) &
    (merged_data['MILEPOST'] <= merged_data['MILEPOST_END'] * (1 + tolerance))
]

# Export
filtered_data_XAI.to_excel('filtered_data_XAI.xlsx', index=False)

# Prediction: Machine-learning
# Classification

Gradient Boosting  >> OK <<

In [ ]:
#ML #Gradient Boosting + SMOTE
# Preparing whole data for training
# Quantifying values in the "DEF_PRTY","TSC_CD" and "DFCT_TYPE" column

filtered_data_grboost = pd.read_excel("filtered_data_XAI.xlsx")
filtered_data_grboost['DEF_PRTY'] = filtered_data_grboost['DEF_PRTY'].replace({'YEL': 0, 'RED': 1})
filtered_data_grboost['TSC_CD'] = filtered_data_grboost['TSC_CD'].replace({'T': 0, 'C': 1})
filtered_data_grboost['DFCT_TYPE'] = filtered_data_grboost['DFCT_TYPE'].replace({'XLEVEL': 1, 'DIP': 2, 'SURFACE': 3})

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

# remove outliers
selected_cols_98 = ['DEF_AMPLTD','DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST', 'TOT_DFLT_MGT']
percentile_98 = filtered_data_grboost[selected_cols_98].quantile(0.98)
filtered_data_grboost = filtered_data_grboost[
    (filtered_data_grboost[selected_cols_98] <= percentile_98).all(axis=1)
]

# Features and target
#X_train = filtered_data_grboost[['MONTH','YEAR','DEF_AMPLTD','DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST', 'TOT_DFLT_MGT']]
X_raw = filtered_data_grboost[['DEF_LGTH','TEST_FSPD','TSC_CD','TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST','TOT_DFLT_MGT']]
#X_raw = filtered_data_grboost[['DEF_LGTH','TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_WEST','TOT_DFLT_MGT']]
Y_train = filtered_data_grboost['DEF_PRTY']

# Apply StandardScaler
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_raw), columns=X_raw.columns)

#  30% test
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)

# Apply SMOTE to training data only
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train, Y_train = smote.fit_resample(X_train, Y_train)

# Gradient Boosting Classifier
model_grboost = GradientBoostingClassifier()

# model training
model_grboost.fit(X_train, Y_train)

# Validation accuracy
accuracy_val_grboost = model_grboost.score(X_val, Y_val)
print("Validation Accuracy:", accuracy_val_grboost)

# k-fold cross-validation
cv_scores_grboost = cross_val_score(model_grboost, X_train, Y_train, cv=5)
print("Cross-Validation Scores:", cv_scores_grboost)
print("Mean CV Accuracy:", np.mean(cv_scores_grboost))

# Evaluattion
Y_pred_grboost = model_grboost.predict(X_test)
accuracy_test_grboost = accuracy_score(Y_test, Y_pred_grboost)
precision_grboost = precision_score(Y_test, Y_pred_grboost)
recall_grboost = recall_score(Y_test, Y_pred_grboost)
f1_grboost = f1_score(Y_test, Y_pred_grboost)
roc_auc_grboost = roc_auc_score(Y_test, Y_pred_grboost)

print("Test Accuracy:", accuracy_test_grboost)
print("Precision:", precision_grboost)
print("Recall:", recall_grboost)
print("F1-Score:", f1_grboost)
print("ROC AUC Score:", roc_auc_grboost)


import joblib
joblib.dump(model_grboost, 'model_grboost.pkl')


In [ ]:
# --- XAI Tools Gragient Boosting ----
# IAL: Immediate action limits
# IL:  Intervention limits

# 1. LIME
import lime
import lime.lime_tabular

explainer_lime_grboost= lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns,
    class_names=['IL','IA'],
     mode='classification')

i = 0  # instance
exp_grboost = explainer_lime_grboost.explain_instance(X_test.iloc[i], model_grboost.predict_proba)
exp_grboost.show_in_notebook(show_table=True)

joblib.dump(exp_grboost, "explainer_lime_grboost.joblib")

# 2. SHAP
import shap

explainer_shap_grboost = shap.Explainer(model_grboost.predict, X_train)
shap_values_grboost = explainer_shap_grboost(X_test[:1000])

shap.plots.waterfall(shap_values_grboost[0])  # Local explanation
shap.plots.beeswarm(shap_values_grboost)      # Global importance
shap.plots.heatmap(shap_values_grboost)
shap.plots.bar(shap_values_grboost)


In [ ]:

# Generate LIME plot
fig_lime = exp_grboost.as_pyplot_figure()
fig_lime.savefig("lime_explanation_grboost.jpeg", dpi=300, bbox_inches='tight')
plt.close(fig_lime)

# Generate SHAP plots
shap.plots.waterfall(shap_values_grboost[0], show=False)
plt.savefig("shap_waterfall_grboost.jpeg", dpi=300, bbox_inches='tight')
plt.close()

shap.plots.beeswarm(shap_values_grboost, show=False)
plt.savefig("shap_beeswarm_grboost.jpeg", dpi=300, bbox_inches='tight')
plt.close()

shap.plots.heatmap(shap_values_grboost, show=False)
plt.savefig("shap_heatmap_grboost.jpeg", dpi=300, bbox_inches='tight')
plt.close()

shap.plots.bar(shap_values_grboost, show=False)
plt.savefig("shap_bar_grboost.jpeg", dpi=300, bbox_inches='tight')
plt.close()


XGBoost >>OK<<

In [ ]:
#ML #XGBoost+SMOTE

import xgboost as xgb

filtered_data_xgboost= pd.read_excel("filtered_data_XAI.xlsx")

# categorical columns to numeric
categorical_columns = ['MONTH','DEF_AMPLTD', 'DEF_LGTH', 'TEST_FSPD', 'TEST_PSPD', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST', 'TOT_DFLT_MGT']
filtered_data_xgboost[categorical_columns] = filtered_data_xgboost[categorical_columns].apply(pd.to_numeric)

categorical_columns = ['DEF_PRTY', 'TSC_CD', 'DFCT_TYPE']
for col in categorical_columns:
    filtered_data_xgboost[col] = filtered_data_xgboost[col].replace({'YEL': 0, 'RED': 1, 'T': 0, 'C': 1, 'XLEVEL': 1, 'DIP': 2, 'SURFACE': 3})

#  remove outliers
selected_cols_98 = ['DEF_AMPLTD','DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST', 'TOT_DFLT_MGT']
percentile_98 = filtered_data_xgboost[selected_cols_98].quantile(0.98)
filtered_data_xgboost = filtered_data_xgboost[
    (filtered_data_xgboost[selected_cols_98] <= percentile_98).all(axis=1)
]

# Features and target
#X_train = filtered_data_xgboost[['MONTH','YEAR','DEF_AMPLTD','DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST', 'TOT_DFLT_MGT']]
X_train =filtered_data_xgboost[['DEF_LGTH','TEST_FSPD','TSC_CD','TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST','TOT_DFLT_MGT']]
#X_train =filtered_data_xgboost[['DEF_LGTH','TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_WEST','TOT_DFLT_MGT']]
Y_train = filtered_data_xgboost['DEF_PRTY']

# and 30 % test sets
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)

# Apply SMOTE to training data only
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train, Y_train = smote.fit_resample(X_train, Y_train)

# XGBoost classifier
xgboost_model = xgb.XGBClassifier()

# Train the modelt
xgboost_model.fit(X_train, Y_train)

# Evaluate the model on the validation set
Y_val_pred = xgboost_model.predict(X_val)
accuracy_val_xgboost = accuracy_score(Y_val, Y_val_pred)
print("Validation Accuracy:", accuracy_val_xgboost)

# Perform k-fold cross-validation
cv_scores_xgboost = cross_val_score(xgboost_model, X_train, Y_train, cv=5)
print("Cross-Validation Scores:", cv_scores_xgboost)
print("Mean CV Accuracy:", np.mean(cv_scores_xgboost))

# Evaluateion
Y_pred = xgboost_model.predict(X_test)
accuracy_test_xgboost = accuracy_score(Y_test, Y_pred)
precision_xgboost = precision_score(Y_test, Y_pred)
recall_xgboost = recall_score(Y_test, Y_pred)
f1_xgboost = f1_score(Y_test, Y_pred)
roc_auc_xgboost = roc_auc_score(Y_test, Y_pred)

print("Test Accuracy:", accuracy_test_xgboost)
print("Precision:", precision_xgboost)
print("Recall:", recall_xgboost)
print("F1-Score:", f1_xgboost)
print("ROC AUC Score:", roc_auc_xgboost)

# Save
joblib.dump(xgboost_model, 'xgboost_model.pkl')

In [ ]:
# --- XAI Tools  XGBoost ---
# IAL: Immediate action limits
# IL:  Intervention limits

# 1. LIME
import lime
import lime.lime_tabular

explainer_lime_xgb = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns,
    class_names=['IL','IA'],
    mode='classification'
)
i = 0  # instance
exp_xgb = explainer_lime_xgb.explain_instance(X_test.iloc[i], xgboost_model.predict_proba)
exp_xgb.show_in_notebook(show_table=True)

joblib.dump(exp_grboost, "explainer_lime_xgb.joblib")

# 2. SHAP
import shap

explainer_shap_xgb = shap.Explainer(xgboost_model, X_train)
shap_values_xgb = explainer_shap_xgb(X_test[:1000])

shap.plots.waterfall(shap_values_xgb[0])  # Local explanation
shap.plots.beeswarm(shap_values_xgb)      # Global importance
shap.plots.bar(shap_values_xgb)
shap.plots.heatmap(shap_values_xgb)


In [ ]:
# Generate LIME plot
fig_lime = exp_grboost.as_pyplot_figure()
fig_lime.savefig("lime_explanation_xgb.jpeg", dpi=300, bbox_inches='tight')
plt.close(fig_lime)

# Save SHAP
shap.plots.beeswarm(shap_values_xgb, show=False)
plt.savefig("shap_beeswarm_xgb.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

shap.plots.waterfall(shap_values_xgb[0], show=False)
plt.savefig("shap_waterfall_xgb.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

shap.plots.bar(shap_values_xgb, show=False)
plt.savefig("shap_bar_xgb.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

shap.plots.heatmap(shap_values_xgb, show=False)
plt.savefig("shap_heatmap_xgb.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

Random Forest >>OK<<

In [ ]:
#ML #Random Forest+SMOTE
from sklearn.ensemble import RandomForestClassifier

# MASK
filtered_data_rndforest = pd.read_excel("filtered_data_XAI.xlsx")

filtered_data_rndforest['DEF_PRTY'] = filtered_data_rndforest['DEF_PRTY'].replace({'YEL': 0, 'RED': 1})
filtered_data_rndforest['TSC_CD'] = filtered_data_rndforest['TSC_CD'].replace({'T': 0, 'C': 1})
filtered_data_rndforest['DFCT_TYPE'] = filtered_data_rndforest['DFCT_TYPE'].replace({'XLEVEL': 1, 'DIP': 2, 'SURFACE': 3})

#  98th percentile to remove outliers
selected_cols_98 = ['DEF_AMPLTD','DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST', 'TOT_DFLT_MGT']
percentile_98 = filtered_data_rndforest[selected_cols_98].quantile(0.98)
filtered_data_rndforest = filtered_data_rndforest[
    (filtered_data_rndforest[selected_cols_98] <= percentile_98).all(axis=1)
]

#  X_train and Y_train
# Features and target
#X_train = filtered_data_rndforest[['MONTH','YEAR','DEF_AMPLTD','DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST', 'TOT_DFLT_MGT']]
X_raw = filtered_data_rndforest[['DEF_LGTH','TEST_FSPD','TSC_CD','TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST','TOT_DFLT_MGT']]
#X_raw = filtered_data_rndforest[['DEF_LGTH','TEST_PSPD', 'DFCT_TYPE','TOT_CAR_WEST','TOT_DFLT_MGT']]
Y_train = filtered_data_rndforest['DEF_PRTY']

# Apply StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw)
X_train = pd.DataFrame(X_scaled, columns=X_raw.columns)  # Preserve column names

# split data 30% test
from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)

# Apply SMOTE to training data only
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train, Y_train = smote.fit_resample(X_train, Y_train)

#  Random Forest Classifier model
model_rndforest = RandomForestClassifier()

# Train
model_rndforest.fit(X_train, Y_train)

# Evaluate the model on the validation set
accuracy_val_rndforest = model_rndforest.score(X_val, Y_val)
print("Validation Accuracy:", accuracy_val_rndforest)

# Perform k-fold cross-validation
cv_scores_rndforest = cross_val_score(model_rndforest, X_train, Y_train, cv=5)
print("Cross-Validation Scores:", cv_scores_rndforest)
print("Mean CV Accuracy:", np.mean(cv_scores_rndforest))

# Evaluate additional metrics on the test set
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
accuracy_rndforest = model_rndforest.score(X_test, Y_test)
Y_pred_rndforest = model_rndforest.predict(X_test)
precision_rndforest = precision_score(Y_test, Y_pred_rndforest)
recall_rndforest = recall_score(Y_test, Y_pred_rndforest)
f1_rndforest = f1_score(Y_test, Y_pred_rndforest)
roc_auc_rndforest = roc_auc_score(Y_test, Y_pred_rndforest)

print("Test Accuracy:", accuracy_rndforest)
print("Precision:", precision_rndforest)
print("Recall:", recall_rndforest)
print("F1-Score:", f1_rndforest)
print("ROC AUC Score:", roc_auc_rndforest)

# Save
joblib.dump(model_rndforest, 'model_rndforest.pkl')

In [ ]:
# --- XAI Tools  RandomForest ---
# IAL: Immediate action limits
# IL:  Intervention limits

# 1. LIME

explainer_lime_rnd = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns,
    class_names=['IL', 'IAL'],
    mode='classification'
)
i = 0  # instance
exp_rnd = explainer_lime_rnd.explain_instance(X_test.iloc[i],model_rndforest.predict_proba)
exp_rnd.show_in_notebook(show_table=True)

joblib.dump(exp_rnd, "explainer_lime_rnd.joblib")

# 2. SHAP

# Create SHAP explainer
explainer_shap_rnd = shap.Explainer(model_rndforest, X_train)

# Compute SHAP values on the test set
shap_values_rnd = explainer_shap_rnd(X_test[:1000], check_additivity=False)  #beacuse of scaling

# Get SHAP values only for class 1 (assuming output is 3D)
shap_values_class1 = shap_values_rnd[:, :, 1]

#Now plot global importance
shap.plots.waterfall(shap_values_class1[0])
shap.plots.beeswarm(shap_values_class1, max_display=10)
shap.plots.bar(shap_values_class1, max_display=10)
shap.plots.heatmap(shap_values_class1)


In [ ]:
# Save LIME explanation as image
fig_lime_rf = exp_rnd.as_pyplot_figure()
fig_lime_rf.savefig("lime_explanation_rndforest.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close(fig_lime_rf)

# Save SHAP beeswarm plot
shap.plots.beeswarm(shap_values_class1, max_display=10, show=False)
plt.savefig("shap_beeswarm_rndforest.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

# Save SHAP waterfall plot for the first instance
shap.plots.waterfall(shap_values_class1[0], show=False)
plt.savefig("shap_waterfall_rndforest.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

# Save SHAP bar plot
shap.plots.bar(shap_values_class1, max_display=10, show=False)
plt.savefig("shap_bar_rndforest.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

# Save SHAP bar plot
shap.plots.heatmap(shap_values_class1, max_display=10, show=False)
plt.savefig("shap_heatmap_rndforest.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

Logistic Regression

In [ ]:
#ML #Logistic Regression+SMOTE

import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

filtered_data_logistic = pd.read_excel("filtered_data_XAI.xlsx")

# Convert categorical columns to numeric
categorical_columns = ['MONTH','DEF_AMPLTD', 'DEF_LGTH', 'TEST_FSPD', 'TEST_PSPD', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST', 'TOT_DFLT_MGT']
filtered_data_logistic[categorical_columns] = filtered_data_logistic[categorical_columns].apply(pd.to_numeric)

categorical_columns = ['DEF_PRTY', 'TSC_CD', 'DFCT_TYPE']
for col in categorical_columns:
    filtered_data_logistic[col] = filtered_data_logistic[col].replace({'YEL': 0, 'RED': 1, 'T': 0, 'C': 1, 'XLEVEL': 1, 'DIP': 2, 'SURFACE': 3})

# Keep only data below the 98th percentile
selected_cols_98 = ['DEF_AMPLTD','DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST', 'TOT_DFLT_MGT']
percentile_98 = filtered_data_logistic[selected_cols_98].quantile(0.98)
filtered_data_logistic = filtered_data_logistic[
    (filtered_data_logistic[selected_cols_98] <= percentile_98).all(axis=1)
]

# Define input features (X) and target variable (Y) for training
#X_train = filtered_data_logistic[['MONTH','YEAR','DEF_AMPLTD','DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST', 'TOT_DFLT_MGT']]
#X_train =filtered_data_logistic[['MONTH','YEAR','DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST']]
X_raw = filtered_data_logistic[['DEF_LGTH','TEST_FSPD','TSC_CD','TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST','TOT_DFLT_MGT']]
#X_raw = filtered_data_logistic[['DEF_LGTH','TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_WEST','TOT_DFLT_MGT']]
Y_train = filtered_data_logistic['DEF_PRTY']

# Apply StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_raw), columns=X_raw.columns)

# Split the data
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)

# Apply SMOTE to training data only
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train, Y_train = smote.fit_resample(X_train, Y_train)

# Logistic Regression classifier
model_logistic = LogisticRegression()

# Train
model_logistic.fit(X_train, Y_train)

# Evaluate the model on the validation set
Y_val_pred = model_logistic.predict(X_val)
accuracy_val_logistic = accuracy_score(Y_val, Y_val_pred)
print("Validation Accuracy:", accuracy_val_logistic)

# Perform k-fold cross-validation
cv_scores_logistic = cross_val_score(model_logistic, X_train, Y_train, cv=5)
print("Cross-Validation Scores:", cv_scores_logistic)
print("Mean CV Accuracy:", np.mean(cv_scores_logistic))

# Evaluate the model on the test set
Y_pred = model_logistic.predict(X_test)
accuracy_test_logistic = accuracy_score(Y_test, Y_pred)
precision_logistic = precision_score(Y_test, Y_pred)
recall_logistic = recall_score(Y_test, Y_pred)
f1_logistic = f1_score(Y_test, Y_pred)
roc_auc_logistic = roc_auc_score(Y_test, Y_pred)

print("Test Accuracy:", accuracy_test_logistic)
print("Precision:", precision_logistic)
print("Recall:", recall_logistic)
print("F1-Score:", f1_logistic)
print("ROC AUC Score:", roc_auc_logistic)

# Save
import joblib
joblib.dump(model_logistic, 'logistic_regression_model.pkl')


In [ ]:
# --- XAI Tools  Logistic Regression ---
# 1. LIME
import lime
import lime.lime_tabular

# LIME expects the unscaled, raw input
explainer_lime_log = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns,
    class_names=['IL', 'IAL'],
    mode='classification'
)
i = 0  # instance
exp_log = explainer_lime_log.explain_instance(X_test.iloc[i], model_logistic.predict_proba)
exp_log.show_in_notebook(show_table=True)

joblib.dump(exp_log, "explainer_lime_log.joblib")

# 2. SHAP
import shap

# Use LinearExplainer for Logistic Regression
explainer_shap_log = shap.Explainer(model_logistic, X_train)
shap_values_log = explainer_shap_log(X_test[:1000])

# Visualize SHAP values
shap.plots.waterfall(shap_values_log[0])  # Local explanation
shap.plots.beeswarm(shap_values_log)      # Global importance
shap.plots.heatmap(shap_values_log)
shap.plots.bar(shap_values_log)

In [ ]:
# Save LIME plot
fig_lime_log = exp_log.as_pyplot_figure()
fig_lime_log.savefig("lime_explanation_logistic.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close(fig_lime_log)

# Save SHAP beeswarm plot
shap.plots.beeswarm(shap_values_log, show=False)
plt.savefig("shap_beeswarm_logistic.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

# Save SHAP waterfall plot
shap.plots.waterfall(shap_values_log[0], show=False)
plt.savefig("shap_waterfall_logistic.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

# Save SHAP bar plot
shap.plots.bar(shap_values_log, show=False)
plt.savefig("shap_bar_logistic.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

# Save SHAP heatmap plot
shap.plots.heatmap(shap_values_log, show=False)
plt.savefig("shap_heatmap_logistic.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

SVM

In [ ]:
#ML #SVM + SMOTE
# Preparing whole data for training
# Quantifying values in the "DEF_PRTY","TSC_CD" and "DFCT_TYPE" column

filtered_data_svm = pd.read_excel("filtered_data_XAI.xlsx")
filtered_data_svm['DEF_PRTY'] = filtered_data_svm['DEF_PRTY'].replace({'YEL': 0, 'RED': 1})
filtered_data_svm['TSC_CD'] = filtered_data_svm['TSC_CD'].replace({'T': 0, 'C': 1})
filtered_data_svm['DFCT_TYPE'] = filtered_data_svm['DFCT_TYPE'].replace({'XLEVEL': 1, 'DIP': 2, 'SURFACE': 3})

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

#   outliers  percentile_9
selected_cols_98 = ['DEF_AMPLTD','DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST', 'TOT_DFLT_MGT']
percentile_98 = filtered_data_svm[selected_cols_98].quantile(0.98)
filtered_data_svm = filtered_data_svm[
    (filtered_data_svm[selected_cols_98] <= percentile_98).all(axis=1)
]

# Features and target
#X_train = filtered_data_svm[['MONTH','YEAR','DEF_AMPLTD','DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST', 'TOT_DFLT_MGT']]
X_raw = filtered_data_svm[['DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST','TOT_DFLT_MGT']]
#X_raw = filtered_data_svm[['DEF_LGTH','TEST_PSPD', 'DFCT_TYPE','TOT_CAR_WEST','TOT_DFLT_MGT']]
Y_train = filtered_data_svm['DEF_PRTY']

# Apply StandardScaler and preserve feature names
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_raw), columns=X_raw.columns)

# Split data to training and testing sets, 30% test
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)

# Apply SMOTE to training data only
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train, Y_train = smote.fit_resample(X_train, Y_train)

# SVM Classifier
model_svm = SVC(probability=True)

# model training
model_svm.fit(X_train, Y_train)

# Validation accuracy
accuracy_val_svm = model_svm.score(X_val, Y_val)
print("Validation Accuracy:", accuracy_val_svm)

# k-fold cross-validation
cv_scores_svm = cross_val_score(model_svm, X_train, Y_train, cv=5)
print("Cross-Validation Scores:", cv_scores_svm)
print("Mean CV Accuracy:", np.mean(cv_scores_svm))

# Evaluate the model
Y_pred_svm = model_svm.predict(X_test)
accuracy_test_svm = accuracy_score(Y_test, Y_pred_svm)
precision_svm = precision_score(Y_test, Y_pred_svm)
recall_svm = recall_score(Y_test, Y_pred_svm)
f1_svm = f1_score(Y_test, Y_pred_svm)
roc_auc_svm = roc_auc_score(Y_test, Y_pred_svm)

print("Test Accuracy:", accuracy_test_svm)
print("Precision:", precision_svm)
print("Recall:", recall_svm)
print("F1-Score:", f1_svm)
print("ROC AUC Score:", roc_auc_svm)

# Save
import joblib
joblib.dump(model_svm, 'model_svm.pkl')

In [ ]:
# --- XAI Tools SVM  ---
# IAL: Immediate action limits
# IL:  Intervention limits

# 1. LIME
import lime
import lime.lime_tabular

explainer_lime_svm= lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns,
    class_names=['IL','IA'],
     mode='classification')

i = 0  # instance number
exp_svm = explainer_lime_svm.explain_instance(X_test.iloc[i], model_svm.predict_proba)
exp_svm.show_in_notebook(show_table=True)

joblib.dump(exp_svm, "lime_explanation_svm.joblib")

# 2. SHAP
import shap

explainer_shap_svm = shap.Explainer(model_svm.predict, X_train)
shap_values_svm = explainer_shap_svm(X_test[:1000]) # reduce X_test[n] to 10 for quick result

shap.plots.waterfall(shap_values_svm[0])  # Local explanation
shap.plots.beeswarm(shap_values_svm)      # Global importance
shap.plots.heatmap(shap_values_svm)
shap.plots.bar(shap_values_svm)


# Saving explanation for furthurte use

# --- Save SHAP Values ---
joblib.dump(shap_values_svm, "shap_values_svm.joblib")  # Already correct


In [ ]:
# --- Load LIME Explanation ---
#exp_svm = joblib.load("lime_explanation_svm.joblib")
#exp_svm.show_in_notebook(show_table=True)  # Regenerate plot

# --- Load SHAP Values ---
#shap_values_svm = joblib.load("shap_values_svm.joblib")
#shap.plots.waterfall(shap_values_svm[0])  # Regenerate SHAP plots

# Save LIME plot
fig_lime_svm = exp_svm.as_pyplot_figure()
fig_lime_svm.savefig("explainer_lime_svm.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close(fig_lime_svm)

# Save SHAP beeswarm plot
shap.plots.beeswarm(shap_values_svm, show=False)
plt.savefig("shap_beeswarm_svm.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

# Save SHAP waterfall plot
shap.plots.waterfall(shap_values_svm[0], show=False)
plt.savefig("shap_waterfall_svm.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

# Save SHAP bar plot
shap.plots.bar(shap_values_svm, show=False)
plt.savefig("shap_bar_svm.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

# Save SHAP heatmap plot
shap.plots.heatmap(shap_values_svm, show=False)
plt.savefig("shap_heatmap_svm.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

Artificial Neural Network

In [ ]:
# ML: Artificial Neural Network with SMOTE, Standardization, and XAI
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load and preprocess the data
filtered_data_nn = pd.read_excel("filtered_data_XAI.xlsx")
filtered_data_nn['DEF_PRTY'] = filtered_data_nn['DEF_PRTY'].replace({'YEL': 0, 'RED': 1})
filtered_data_nn['TSC_CD'] = filtered_data_nn['TSC_CD'].replace({'T': 0, 'C': 1})
filtered_data_nn['DFCT_TYPE'] = filtered_data_nn['DFCT_TYPE'].replace({'XLEVEL': 1, 'DIP': 2, 'SURFACE': 3})

# Keep only data below the 80th percentile for each numeric column to remove outliers
selected_cols_98 = ['DEF_AMPLTD','DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST', 'TOT_DFLT_MGT']
percentile_98 = filtered_data_nn[selected_cols_98].quantile(0.98)
filtered_data_nn = filtered_data_nn[
    (filtered_data_nn[selected_cols_98] <= percentile_98).all(axis=1)
]

# Define features and target
feature_names = ['DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST','TOT_DFLT_MGT']
#feature_names = ['DEF_LGTH','TEST_PSPD', 'DFCT_TYPE','TOT_CAR_WEST','TOT_DFLT_MGT']
X_train = filtered_data_nn[feature_names]
Y_train = filtered_data_nn['DEF_PRTY']

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Split into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)

# Apply SMOTE to the training set
smote = SMOTE(random_state=42)
X_train_resampled, Y_train_resampled = smote.fit_resample(X_train, Y_train)

# Initialize and train MLPClassifier
model_nn = MLPClassifier(hidden_layer_sizes=(35), activation='relu', solver='adam', max_iter=1000, random_state=42)
model_nn.fit(X_train_resampled, Y_train_resampled)

# Validation accuracy
accuracy_val_nn = model_nn.score(X_val, Y_val)
print("Validation Accuracy:", accuracy_val_nn)

# Cross-validation scores
cv_scores_nn = cross_val_score(model_nn, X_train_resampled, Y_train_resampled, cv=5)
print("Cross-Validation Scores:", cv_scores_nn)
print("Mean CV Accuracy:", np.mean(cv_scores_nn))

# Evaluate on test set
accuracy_test_nn = model_nn.score(X_test, Y_test)
precision_nn = precision_score(Y_test, model_nn.predict(X_test))
recall_nn = recall_score(Y_test, model_nn.predict(X_test))
f1_nn = f1_score(Y_test, model_nn.predict(X_test))
roc_auc_nn = roc_auc_score(Y_test, model_nn.predict(X_test))

print("\nTest Performance Metrics:")
print("Test Accuracy:", accuracy_test_nn)
print("Precision:", precision_nn)
print("Recall:", recall_nn)
print("F1-Score:", f1_nn)
print("ROC AUC Score:", roc_auc_nn)

# Save model
joblib.dump(model_nn, 'neural_network_model.pkl')


In [ ]:
# --- XAI Tools  Neural Network---
# IAL: Immediate action limits
# IL: Intervention limits

# 1. LIME
import lime
import lime.lime_tabular

explainer_lime_nn = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train_resampled),
    feature_names=feature_names,
    class_names=['IL','IA'],
    mode='classification')

i = 0  # instance number
exp_nn = explainer_lime_nn.explain_instance(X_test[i], model_nn.predict_proba)
exp_nn.show_in_notebook(show_table=True)

joblib.dump(exp_svm, "explainer_lime_nn.joblib")

# 2. SHAP
import shap

explainer_shap_nn = shap.KernelExplainer(model_nn.predict, X_train[:1000])
shap_values_array = explainer_shap_nn.shap_values(X_test[:1000])
shap_values_nn = shap.Explanation(values=shap_values_array,
                                  base_values=np.repeat(explainer_shap_nn.expected_value, X_test[:1000].shape[0]),
                                  data=X_test[:1000],
                                  feature_names=feature_names)

# Visualize SHAP values
shap.plots.waterfall(shap_values_nn[0])  # Local explanation
shap.plots.beeswarm(shap_values_nn)      # Global importance
shap.plots.heatmap(shap_values_nn)
shap.plots.bar(shap_values_nn)

In [ ]:
# Save LIME explanation as image
fig_lime_nn = exp_nn.as_pyplot_figure()
fig_lime_nn.savefig("lime_explanation_nn.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close(fig_lime_nn)

# Save SHAP beeswarm plot
shap.plots.beeswarm(shap_values_nn, show=False)
plt.savefig("shap_beeswarm_nn.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()


# Save SHAP waterfall plot
shap.plots.waterfall(shap_values_nn[0], show=False)
plt.savefig("shap_waterfall_nn.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

# Save SHAP bar plot
shap.plots.bar(shap_values_nn, show=False)
plt.savefig("shap_bar_nn.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

# Save SHAP heatmap plot
shap.plots.heatmap(shap_values_nn, show=False)
plt.savefig("shap_heatmap_nn.jpeg", format="jpeg", dpi=300, bbox_inches='tight')
plt.close()

In [ ]:
#ML accuracy & validation results :

print("Gradient Boosting Model:")
print("Validation Accuracy:", accuracy_test_grboost)
print("Cross-Validation Scores:", cv_scores_grboost)
print("Mean CV Accuracy:", np.mean(cv_scores_grboost))
print("Test Accuracy:", accuracy_test_grboost)
print("Precision:", precision_grboost)
print("Recall:", recall_grboost)
print("F1-Score:", f1_grboost)
print("ROC AUC Score:", roc_auc_grboost)
print()

print("XGBOOST Model:")
print("Validation Accuracy:", accuracy_test_xgboost)
print("Cross-Validation Scores:", cv_scores_xgboost)
print("Mean CV Accuracy:", np.mean(cv_scores_xgboost))
print("Test Accuracy:", accuracy_test_xgboost)
print("Precision:", precision_xgboost)
print("Recall:", recall_xgboost)
print("F1-Score:", f1_xgboost)
print("ROC AUC Score:", roc_auc_xgboost)
print()

print("Random forest Model:")
print("Validation Accuracy:", accuracy_rndforest)
print("Cross-Validation Scores:", cv_scores_rndforest)
print("Mean CV Accuracy:", np.mean(cv_scores_rndforest))
print("Test Accuracy:", accuracy_rndforest)
print("Precision:", precision_rndforest)
print("Recall:", recall_rndforest)
print("F1-Score:", f1_rndforest)
print("ROC AUC Score:", roc_auc_rndforest)
print()

print("Logistic Regression Model:")
print("Validation Accuracy:", accuracy_val_logistic)
print("Cross-Validation Scores:", cv_scores_logistic)
print("Mean CV Accuracy:", np.mean(cv_scores_logistic))
print("Test Accuracy:", accuracy_test_logistic)
print("Precision:", precision_logistic)
print("Recall:", recall_logistic)
print("F1-Score:", f1_logistic)
print("ROC AUC Score:", roc_auc_logistic)
print()

print("Support Vector Machine Model:")
print("Validation Accuracy:", accuracy_val_svm)
print("Cross-Validation Scores:", cv_scores_svm)
print("Mean CV Accuracy:", np.mean(cv_scores_svm))
print("Test Accuracy:", accuracy_test_svm)
print("Precision:", precision_svm)
print("Recall:", recall_svm)
print("F1-Score:", f1_svm)
print("ROC AUC Score:", roc_auc_svm)
print()

print("Neural Network Model:")
print("Validation Accuracy:", accuracy_test_nn)
print("Cross-Validation Scores:", cv_scores_nn)
print("Mean CV Accuracy:", np.mean(cv_scores_nn))
print("Test Accuracy:", accuracy_test_nn)
print("Precision:", precision_nn)
print("Recall:", recall_nn)
print("F1-Score:", f1_nn)
print("ROC AUC Score:", roc_auc_nn)
print()

In [ ]:
# table for comparing ML:

# Data for the table
ml_table = [
    ["Logistic Regression", accuracy_val_logistic, np.mean(cv_scores_logistic), accuracy_test_logistic, precision_logistic, recall_logistic, f1_logistic, roc_auc_logistic],
    ["Gradient Boosting", accuracy_val_grboost, np.mean(cv_scores_grboost), accuracy_test_grboost, precision_grboost, recall_grboost, f1_grboost, roc_auc_grboost],
    ["XGBOOST", accuracy_val_xgboost, np.mean(cv_scores_xgboost), accuracy_test_xgboost, precision_xgboost, recall_xgboost, f1_xgboost, roc_auc_xgboost],
    ["Random Forest", accuracy_val_rndforest, np.mean(cv_scores_rndforest), accuracy_rndforest, precision_rndforest, recall_rndforest, f1_rndforest, roc_auc_rndforest],
    ["SVM", accuracy_val_svm,  np.mean(cv_scores_svm), accuracy_test_svm, precision_svm, recall_svm, f1_svm, roc_auc_svm],
    ["Neural Network", accuracy_val_nn,np.mean(cv_scores_nn), accuracy_test_nn, precision_nn, recall_nn, f1_nn, roc_auc_nn],]

# Column headers
headers = ["Model", "Validation Accuracy", "Mean Cross-Validation Score", "Test Accuracy", "Precision", "Recall", "F1-Score", "ROC AUC Score"]

# Convert the ML table into a DataFrame
ml_table = pd.DataFrame(ml_table, columns=headers)

display(ml_table)
ml_table.to_excel("ml_table.xlsx", index=False)

In [ ]:

display('Logistic Regression')

shap.plots.beeswarm(shap_values_log)

display('GRboost')

shap.plots.beeswarm(shap_values_grboost)

display('XGboost')

shap.plots.beeswarm(shap_values_xgb)

display('RNDforest')

shap.plots.beeswarm(shap_values_class1)

display('Support Vector Machine')

shap.plots.beeswarm(shap_values_svm)

display('Neural Network')

shap.plots.beeswarm(shap_values_nn[:1000])  #Always check the number [:100]

shap_values_nn = shap.Explanation(
    values=shap_values_array,
    base_values=np.repeat(explainer_shap_nn.expected_value, X_test[:1000].shape[0]),
    data=X_test[:1000],
    feature_names=feature_names
)
import shap
import matplotlib.pyplot as plt
from PIL import Image

# Titles and corresponding SHAP values
titles = [
    'Logistic Regression',
    'GRboost',
    'XGboost',
    'Random Forest',
    'Support Vector Machine',
    'Neural Network'
]

shap_values_list = [
    shap_values_log,
    shap_values_grboost,
    shap_values_xgb,
    shap_values_class1,     # or shap_values_rnd if preferred
    shap_values_svm,
    shap_values_nn
]


# Save each SHAP beeswarm plot individually
filepaths = []
for title, shap_vals in zip(titles, shap_values_list):
    shap.summary_plot(shap_vals, show=False, plot_type='dot', plot_size=(12, 6))
    fig = plt.gcf()
    fig.suptitle(title, fontsize=16)
    filepath = f"{title.replace(' ', '_')}_shap.png"
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
    filepaths.append(filepath)
    plt.close()

# Combine saved images vertically into one
images = [Image.open(fp) for fp in filepaths]
total_height = sum(img.height for img in images)
max_width = max(img.width for img in images)
combined = Image.new('RGB', (max_width, total_height), (255, 255, 255))

y_offset = 0
for img in images:
    combined.paste(img, (0, y_offset))
    y_offset += img.height

combined.save("SHAP_beeswarm_all_models.png")

In [ ]:
# Create folder for temporary plots
os.makedirs("lime_temp_plots", exist_ok=True)

# Load LIME explanations
lime_exps = {
    "Logistic Regression": joblib.load("explainer_lime_log.joblib"),
    "Random Forest": joblib.load("explainer_lime_rnd.joblib"),
    "Gradient Boosting": joblib.load("explainer_lime_grboost.joblib"),
    "XGBoost": joblib.load("explainer_lime_xgb.joblib"),
    "SVM": joblib.load("lime_explanation_svm.joblib"),
    "Neural Network": joblib.load("explainer_lime_nn.joblib")
}

# Instance setup
instance_index = 10

# Ensure instance_data is 1D array
instance_data = X_train[instance_index]
instance_data = instance_data.flatten() if instance_data.ndim > 1 else instance_data

# Access class label
class_label = Y_train.iloc[instance_index] if hasattr(Y_train, "iloc") else Y_train[instance_index]

# Feature names (adjust to match your data)
feature_names = ['DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST','TOT_DFLT_MGT']

# Validate length
assert len(instance_data) == len(feature_names), "Mismatch between instance and feature names"

# Build feature table
feature_table = pd.DataFrame({
    'Feature': feature_names,
    'Value': instance_data
})

# --- Plot setup ---
fig = plt.figure(figsize=(8, 26))
gs = fig.add_gridspec(7, 2, height_ratios=[0.9] + [1]*6)

# Row 0: Table at top
ax_table = fig.add_subplot(gs[0, :])
ax_table.axis("off")
table = ax_table.table(
    cellText=feature_table.values,
    colLabels=feature_table.columns,
    loc='center',
    cellLoc='center'
)
table.auto_set_font_size(False)
table.set_fontsize(8)
table.scale(1.1, 0.6)
ax_table.set_title(f"Instance {instance_index} – True Class: {class_label}", fontsize=14, pad=12)

# Rows 1–6: LIME plots
for i, (model_name, exp) in enumerate(lime_exps.items()):
    ax = fig.add_subplot(gs[i // 2 + 1, i % 2])

    # Generate LIME plot and customize font sizes
    fig_lime = exp.as_pyplot_figure(label=1)
    ax_lime = fig_lime.axes[0]
    ax_lime.tick_params(axis='both', labelsize=12)
    ax_lime.set_xlabel(ax_lime.get_xlabel(), fontsize=12)
    ax_lime.set_ylabel(ax_lime.get_ylabel(), fontsize=12)
    fig_lime.tight_layout()

    # Save and reinsert into the main grid
    image_path = f"lime_temp_plots/{model_name.replace(' ', '_')}.png"
    fig_lime.savefig(image_path, bbox_inches='tight')
    plt.close(fig_lime)

    img = plt.imread(image_path)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title(model_name, fontsize=12)

# Final layout
plt.suptitle(f"LIME Explanations for Instance {instance_index}", fontsize=30, y=0.96)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig("lime_comparison_grid_with_top_table.png", dpi=300)
plt.show()



In [ ]:
# Instance display



feature_names = ['DEF_LGTH', 'TSC_CD', 'TEST_FSPD', 'TEST_PSPD',
                 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_DFLT_MGT']

# Extract instance
instance_o = filtered_data_grboost.iloc[instance_index][feature_names]

#  table
instance_table = pd.DataFrame(instance_o).T  # Transpose to show as single row table

print(instance_table.to_string(index=False))  # index=False removes row numbering

SVM Second run

In [ ]:
#ML #SVM + SMOTE   SECOND RUN WITHOUT 'TOT_CAR_EAST' and 'TSC_CD'
# Preparing whole data for training
# Quantifying values in the "DEF_PRTY","TSC_CD" and "DFCT_TYPE" column

filtered_data_svm2 = pd.read_excel("filtered_data_XAI.xlsx")
filtered_data_svm2['DEF_PRTY'] = filtered_data_svm2['DEF_PRTY'].replace({'YEL': 0, 'RED': 1})
filtered_data_svm2['TSC_CD'] = filtered_data_svm2['TSC_CD'].replace({'T': 0, 'C': 1})
filtered_data_svm2['DFCT_TYPE'] = filtered_data_svm2['DFCT_TYPE'].replace({'XLEVEL': 1, 'DIP': 2, 'SURFACE': 3})

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

#   outliers  percentile_9
selected_cols_98 = ['DEF_AMPLTD','DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST', 'TOT_DFLT_MGT']
percentile_98 = filtered_data_svm2[selected_cols_98].quantile(0.98)
filtered_data_svm2 = filtered_data_svm2[
    (filtered_data_svm2[selected_cols_98] <= percentile_98).all(axis=1)
]

# Features and target
#X_train = filtered_data_svm2[['MONTH','YEAR','DEF_AMPLTD','DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST', 'TOT_TRN_EAST', 'TOT_TRN_WEST', 'TOT_DFLT_MGT']]
#X_raw = filtered_data_svm2[['DEF_LGTH','TSC_CD','TEST_FSPD', 'TEST_PSPD', 'DFCT_TYPE', 'TOT_CAR_EAST', 'TOT_CAR_WEST','TOT_DFLT_MGT']]
X_raw = filtered_data_svm2[['DEF_LGTH','TEST_PSPD', 'DFCT_TYPE','TOT_CAR_WEST','TOT_DFLT_MGT']]
Y_train = filtered_data_svm2['DEF_PRTY']

# Apply StandardScaler and preserve feature names
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_raw), columns=X_raw.columns)

# Split data to training and testing sets, 30% test
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)

# Apply SMOTE to training data only
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train, Y_train = smote.fit_resample(X_train, Y_train)

# SVM Classifier
model_svm2 = SVC(probability=True)

# model training
model_svm2.fit(X_train, Y_train)

# Validation accuracy
accuracy_val_svm2 = model_svm2.score(X_val, Y_val)
print("Validation Accuracy:", accuracy_val_svm2)

# k-fold cross-validation
cv_scores_svm2 = cross_val_score(model_svm2, X_train, Y_train, cv=5)
print("Cross-Validation Scores:", cv_scores_svm2)
print("Mean CV Accuracy:", np.mean(cv_scores_svm2))

# Evaluate the model
Y_pred_svm2 = model_svm2.predict(X_test)
accuracy_test_svm2 = accuracy_score(Y_test, Y_pred_svm2)
precision_svm2 = precision_score(Y_test, Y_pred_svm2)
recall_svm2 = recall_score(Y_test, Y_pred_svm2)
f1_svm2 = f1_score(Y_test, Y_pred_svm2)
roc_auc_svm2 = roc_auc_score(Y_test, Y_pred_svm2)

print("Test Accuracy:", accuracy_test_svm2)
print("Precision:", precision_svm2)
print("Recall:", recall_svm2)
print("F1-Score:", f1_svm2)
print("ROC AUC Score:", roc_auc_svm2)

# Save
import joblib
joblib.dump(model_svm2, 'model_svm2.pkl')


In [ ]:
# --- XAI Tools SVM  ---
# IAL: Immediate action limits
# IL:  Intervention limits

# 1. LIME
import lime
import lime.lime_tabular

explainer_lime_svm2= lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns,
    class_names=['IL','IA'],
     mode='classification')

i = 0  # instance number
exp_svm2 = explainer_lime_svm2.explain_instance(X_test.iloc[i], model_svm2.predict_proba)
exp_svm2.show_in_notebook(show_table=True)

joblib.dump(exp_svm2, "lime_explanation_svm2.joblib")

# 2. SHAP
import shap

explainer_shap_svm2 = shap.Explainer(model_svm2.predict, X_train)
shap_values_svm2 = explainer_shap_svm2(X_test[:10]) # increase X_test[n] to 1000 for better result

#shap.plots.waterfall(shap_values_svm2[0])  # Local explanation
#shap.plots.beeswarm(shap_values_svm2)      # Global importance
shap.plots.heatmap(shap_values_svm2)
#shap.plots.bar(shap_values_svm2)


# Saving explanation for furthurte use

# --- Save SHAP Values ---
joblib.dump(shap_values_svm2, "shap_values_svm2.joblib")

In [ ]:
# --- Load LIME Explanation ---
#exp_svm2 = joblib.load("lime_explanation_svm2.joblib")
#exp_svm2.show_in_notebook(show_table=True)  # Regenerate plot

# --- Load SHAP Values ---
#shap_values_svm2 = joblib.load("shap_values_svm2.joblib")
#shap.plots.waterfall(shap_values_svm2[0])  # Regenerate SHAP plots

# Save LIME plot
fig_lime_svm2 = exp_svm2.as_pyplot_figure()
fig_lime_svm2.savefig("explainer_lime_svm2.jpeg", format="jpeg", dpi=600, bbox_inches='tight')
plt.close(fig_lime_svm2)

# Save SHAP beeswarm plot
shap.plots.beeswarm(shap_values_svm2, show=False)
plt.savefig("shap_beeswarm_svm2.jpeg", format="jpeg", dpi=600, bbox_inches='tight')
plt.close()

# Save SHAP waterfall plot
shap.plots.waterfall(shap_values_svm2[0], show=False)
plt.savefig("shap_waterfall_svm2.jpeg", format="jpeg", dpi=600, bbox_inches='tight')
plt.close()

# Save SHAP bar plot
shap.plots.bar(shap_values_svm2, show=False)
plt.savefig("shap_bar_svm2.jpeg", format="jpeg", dpi=600, bbox_inches='tight')
plt.close()

# Save SHAP heatmap plot
shap.plots.heatmap(shap_values_svm2, show=False)
plt.savefig("shap_heatmap_svm2.jpeg", format="jpeg", dpi=600, bbox_inches='tight')
plt.close()